### Generate labels for energy table

In [25]:
import pickle
import numpy as np

NRR_chem_list = [
             ['N2_g'],
             ['N2_s'],
             ['N2H_s'],
             ['N2H2_s'],
             ['H2NNH_s'],
             ['H2NNH2_s'],
             ['NH2_s','NH3_s'],
             ['2NH3_s'],
             ['2NH3_g'],]
         
labels_NRR = ['N$_{2}$(g)',
 'N$_{2}$*',
 'N$_{2}$H*',
 'N$_{2}$H$_{2}$*',
 'H$_{2}$NNH*',
 'H$_{2}$NNH$_{2}$*',
 'NH$_{2}$* + NH$_{3}$*',
 '$_{2}$NH$_{3}$*',
 '$_{2}$NH$_{3}$(g)']
              
SI_ingredients = set()
SI_labels = {}

def flatten(l):
    if type(l)!=list:
        return l
    elif len(l)==0:
        return []
    if type(l[0])!=list:
        if l[0]=='CH2N2OH_s':
            return ['CH2OHN2_s'] + flatten(l[1:])
        else:
            return [l[0]] + flatten(l[1:])
    else:
        return flatten(l[0]) + flatten(l[1:])
    
def to_latex_label(name):
    if name =='CH2OHN2_s':
        name = 'CH2N2OH_s'
    for i in range(0,9):
        name = name.replace(str(i), '$_{'+str(i)+'}$')
    name = name.replace('_s', ' at top active site')
    name = name.replace('_g', ' gas')
    return name

for list_of_list in NRR_chem_list:
    l = flatten(list_of_list)
    SI_ingredients = set.union(set(l),SI_ingredients)
SI_ingredients.remove('2NH3_g')
SI_ingredients.remove('N2_g')

SI_ingredients.remove('2NH3_s')
(sorted(SI_ingredients) )

for label in SI_ingredients:
    latex_label = to_latex_label(label)
    SI_labels[label] = latex_label


### SI energy table

In [26]:
f = open('SI/BEEF_free_energy.pkl','rb')
beef_free_eng = pickle.load(f)

f = open('SI/HSE_free_energy.pkl','rb')
hse_free_eng = pickle.load(f)

f = open('SI/BEEF_pathway_ensemble.pkl','rb')
ensemble = pickle.load(f)


### generate 'SI/energy-table.tex'

In [27]:
cap = 'Energies of all species used relative to the slab, H$_2$, N$_2$ and O$_2$ at 0K (ZPE included)'
f = open('SI/energy-table.tex','w+')
f.write('\\clearpage\n\\begin{center}\n\\renewcommand*{\\arraystretch}{1.3}\n\\begin{longtable}{|m{5.8cm} |m{2.9cm}|m{2.9cm} |m{5.5cm} |} \n\caption{'+cap+'}\n\label{tab:energies}\n\endfirsthead\n\endhead\n\hline\n')
f.write('compound & HSE06 energy & BEEF energy  & ensemble standard deviation \\\ \hline\ \n')
# potential_energy_dictionay
n=0

for item in SI_ingredients:
    if '_g' in item:
        continue
    print(SI_labels[item]) 
    name = item.split('_')[0]
    n+=1
    f.write(SI_labels[item]+'&'+str(np.round(hse_free_eng[item],2))+'&'+str(np.round(beef_free_eng[item],2))+'&'+str(np.round((ensemble[item]['std']),2))+  '\\\  \n')
for item in SI_ingredients:
    if '_g' in item:
        print(SI_labels[item]) 
        name = item.split('_')[0]
        n+=1
        f.write(SI_labels[item]+'&'+str(np.round(hse_free_eng[item],2))+'&'+str(np.round(beef_free_eng[item],2))+'&'+str(np.round((ensemble[item]['std']),2))+  '\\\  \n')

    
f.write(' \hline \end{longtable}\n\end{center}\n')
f.close()

NH$_{3}$ at top active site
NH$_{2}$ at top active site
N$_{2}$H at top active site
N$_{2}$H$_{2}$ at top active site
H$_{2}$NNH$_{2}$ at top active site
H$_{2}$NNH at top active site
N$_{2}$ at top active site


### generate 'vib-table.tex'

In [45]:
f = open('SI/species_definitions.pkl', 'rb')
species_definitions =  pickle.load(f)
vib_dictionary = {}
for item in SI_ingredients:
    if item == 'H_g':
        continue
    freq = (species_definitions[item]['frequencies'])
    new_freq = []
    for v in freq:
        new_freq.append(str(round(v*1e3,1)))
    frequencies = '\parbox[t]{6cm}{'
    max_freqs = 3
    while new_freq:
        freq_subset = [new_freq.pop(0) for i in range(0,max_freqs) if new_freq]
        frequencies += ', '.join(freq_subset)+r'\\'
#         print(frequencies)
    frequencies = frequencies[:-2] + '}'
    frequencies
    vib_dictionary[item]=frequencies
# vib_dictionary

In [47]:
# f = open('SI/converged_bulk_vib.pkl','rb')
# ads_slab_vib_dictionary = pickle.load(f)['ads_slab']


cap = 'Vibrational frequencies of all species'
f = open('SI/vibration-table.tex','w+')
f.write('\\clearpage\n\\begin{center}\n\\renewcommand*{\\arraystretch}{2}\n\\begin{longtable}{|m{6cm} |m{6cm} |} \n\caption{'+cap+'}\n\label{tab:energies}\n\endfirsthead\n\endhead\n\hline\n')
f.write('compound & Vibrational Frequencies (cm$^{-1}$) \\\ \hline\ \n')
# potential_energy_dictionay
n=0

for item in SI_ingredients:
    if '_g' in item and item != 'H_g':
#         continue
        print(SI_labels[item]) 
        name = item.split('_')[0]
        n+=1
        f.write(SI_labels[item]+'&'+vib_dictionary[item]+'\\\ \hline \\n ')
for item in SI_ingredients:
    if not '_g' in item:
        print(SI_labels[item]) 
        name = item.split('_')[0]
        n+=1
        f.write(SI_labels[item]+'&'+vib_dictionary[item]+'\\\ \hline \\n ')
        
f.write(' \end{longtable}\n\end{center}\n')
f.close()


NH$_{3}$ at top active site
NH$_{2}$ at top active site
N$_{2}$H at top active site
N$_{2}$H$_{2}$ at top active site
H$_{2}$NNH$_{2}$ at top active site
H$_{2}$NNH at top active site
N$_{2}$ at top active site


### generate surface pic from 'converged.traj'

In [54]:
from ase import io
import os
kwargs = {
  #  'rotation'      : rot, # text string with rotation (default='' )
    'radii'         : .85, # float, or a list with one float per atom
    'colors'        : None,# List: one (r, g, b) tuple per atom
    'show_unit_cell': 0,   # 0, 1, or 2 to not show, show, and show all of cell
    
    }

# Extra kwargs only available for povray (All units in angstrom)
kwargs.update({
    'run_povray'   : True, # Run povray or just write .pov + .ini files
#     'display'      : False,# Display while rendering
#     'pause'        : True, # Pause when done rendering (only if display)
    'transparent'  : False,# Transparent background
    'canvas_width' : None, # Width of canvas in pixels
    'canvas_height': 750, # Height of canvas in pixels 
    'camera_dist'  : 50.,  # Distance from camera to front atom
    'image_plane'  : None, # Distance from front atom to image plane
    'camera_type'  : 'perspective', # perspective, ultra_wide_angle
    'point_lights' : [],             # [[loc1, color1], [loc2, color2],...]
    'area_light'   : [(2., 3., 40.), # location
                      'White',       # color
                      .7, .7, 3, 3], # width, height, Nlamps_x, Nlamps_y
    'background'   : 'White',        # color
    'textures'     : None, # Length of atoms list of texture names
    'celllinewidth': 0.1,  # Radius of the cylinders representing the cell
    
    })


def make_pics(path,fancy=True):
    name=path.split('/')[-1]
    print(name)
    slb = io.read(path+'/converged.traj')
    if not os.path.exists('SI/surface_pic/'):
        os.mkdir('SI/surface_pic/')
    if fancy == True:
        io.write('SI/surface_pic/'+name+'-top.pov',slb,**kwargs)
        io.write('SI/surface_pic/'+name+'-xside.pov',slb,rotation='-90x',**kwargs)
        print('lol')
    else:
        io.write('SI/surface_pic/'+name+'-top.png',slb)
        io.write('SI/surface_pic/'+name+'-xside.png',slb,rotation='-90x')
 
for label in SI_ingredients:
    if '_g' not in label:
        path = 'SI/BEEF_converged_bulk_spin_pol/ads_slab/' + label.split('_')[0]
        print(path)
        make_pics(path,fancy=False)
        
# path = 'SI/BEEF_converged_bulk_spin_pol/ads_slab'
# make_pics(path,fancy=False)


SI/BEEF_converged_bulk_spin_pol/ads_slab/NH3
NH3
SI/BEEF_converged_bulk_spin_pol/ads_slab/NH2
NH2
SI/BEEF_converged_bulk_spin_pol/ads_slab/N2H
N2H
SI/BEEF_converged_bulk_spin_pol/ads_slab/N2H2
N2H2
SI/BEEF_converged_bulk_spin_pol/ads_slab/H2NNH2
H2NNH2
SI/BEEF_converged_bulk_spin_pol/ads_slab/H2NNH
H2NNH
SI/BEEF_converged_bulk_spin_pol/ads_slab/N2
N2


### generate surface pic table

In [60]:
cap = 'Calculated equilibrium geometries for all species.'
f = open('SI/pic-table.tex','w')
f.write('\\clearpage\n\\begin{center}\n\\begin{longtable}{| m{4.5cm} | m{5cm} | m{4.5cm} |}\caption{'+cap+'}\n\label{tab:pic}\n\endfirsthead\n\endhead\n\hline\n')
f.write('compound & Top View & Side View')

for item in SI_ingredients:
    if not '_g' in item:
        name = item.split('_')[0]
        f.write(' \\\ \hline\ \n'+SI_labels[item]+'&'+'\makecell{  \includegraphics[width=0.30\\textwidth, height=38mm]{surface_pathway/'+name+'-top.png}}'
       +'&'+'\makecell{  \includegraphics[width=0.15\\textwidth, height=85mm]{surface_pathway/'+name+'-xside.png}}'
       )
        
f.write(' \\\ \hline \end{longtable}\n\end{center}')

# SI_ingredients
# ACS_SI_ingredients


39